# RAGEx for RAG

basierend auf: https://dl.acm.org/doi/pdf/10.1145/3626772.3657660

In [1]:
import sys, json
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [2]:
from src.modules.rag_ex_explainable import RAGExExplainable
from src.modules.cot_explainable import _format_documents
from src.modules.rag_engine import RAGEngine
from src.modules.llm_client import LLMClient
from src.modules.data_loader_single_hop import BoolQDataLoader

### Real data example

In [3]:
_dataLoader = BoolQDataLoader()
documents = _dataLoader.setup()

In [4]:
ragEngine = RAGEngine(persist_dir="../data/vector_db_bool")
ragEngine.setup(documents=documents)

Loading existing vector store from ../data/vector_db_bool...
RagEngine ready.


In [9]:
client = LLMClient(provider="ollama", model_name="deepseek-r1:1.5b")

question = "What color is Tom's car? Only answer with the solution"
#
#documents = ragEngine.retrieve_documents(question)
#documents = [documents[0]]
#
#print(documents)
#context = _format_documents(documents, from_query=question)
context = 'Tom drives a red car. Max has a blue bike.'
message = client._create_final_answer_prompt(question, context)

llm = client.get_llm()
final_answer_response = llm.invoke(message)
final_answer = final_answer_response.content.strip()

print(final_answer)

explainer = RAGExExplainable(llm_client=client)
result = explainer.explain(query=question, answer=final_answer, context=context)

Connecting to local Ollama (deepseek-r1:1.5b)...
Tom's car is red.
Pertubating 0 of 10
Pertubating 2 of 10
Pertubating 4 of 10
Pertubating 6 of 10
Pertubating 8 of 10
Pertubating 10 of 10
Pertubating 12 of 10
Pertubating 14 of 10
Pertubating 16 of 10
Pertubating 18 of 10


In [10]:
print(result)

{'baseline_answer': "Tom's car is red.", 'context': 'Tom drives a red car. Max has a blue bike.', 'results': [{'token': 'Tom', 'token_index': 0, 'importance_raw': 0.5542183534469012, 'details': [{'strategy': 'leave_one_out', 'levenshtein_similarity': 0.04404145077720212, 'semantic_similarity': 0.71360844373703, 'combined_similarity': 0.4457816465530989, 'importance_raw': 0.5542183534469012}], 'importance': 0.9641023822383055}, {'token': 'drives', 'token_index': 2, 'importance_raw': 0.0, 'details': [{'strategy': 'leave_one_out', 'levenshtein_similarity': 1.0, 'semantic_similarity': 1.0, 'combined_similarity': 1.0, 'importance_raw': 0.0}], 'importance': 0.0}, {'token': 'a', 'token_index': 4, 'importance_raw': 0.0, 'details': [{'strategy': 'leave_one_out', 'levenshtein_similarity': 1.0, 'semantic_similarity': 1.0, 'combined_similarity': 1.0, 'importance_raw': 0.0}], 'importance': 0.0}, {'token': 'red', 'token_index': 6, 'importance_raw': 0.2331626854444805, 'details': [{'strategy': 'leave